In [1]:
pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.2 MB/s eta 0:00:00


In [2]:
pip install xgboost

In [3]:
import optuna
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [4]:
df=pd.read_csv("https://raw.githubusercontent.com/sunnysavita10/svm_kernel/main/Admission_Prediction.csv")

In [5]:
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,NaN,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         500 non-null    int64  
 1   GRE Score          485 non-null    float64
 2   TOEFL Score        490 non-null    float64
 3   University Rating  485 non-null    float64
 4   SOP                500 non-null    float64
 5   LOR                500 non-null    float64
 6   CGPA               500 non-null    float64
 7   Research           500 non-null    int64  
 8   Chance of Admit    500 non-null    float64
dtypes: float64(7), int64(2)
memory usage: 35.3 KB


In [7]:
df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,485.000000,490.000000,485.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.558763,107.187755,3.121649,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.274704,6.112899,1.146160,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


In [8]:
df.isnull().sum()

Serial No.            0
GRE Score            15
TOEFL Score          10
University Rating    15
SOP                   0
LOR                   0
CGPA                  0
Research              0
Chance of Admit       0
dtype: int64

In [9]:
df['GRE Score']=df['GRE Score'].fillna(df['GRE Score'].median())
df['TOEFL Score']=df['TOEFL Score'].fillna(df['TOEFL Score'].median())
df['University Rating']=df['University Rating'].fillna(df['University Rating'].median())

In [10]:
df.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [11]:
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,317.0,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65


In [12]:
X=df.drop(['Serial No.','Chance of Admit'],axis=1)

In [13]:
y=df['Chance of Admit']

In [14]:
X

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337.0,118.0,4.0,4.5,4.5,9.65,1
1,324.0,107.0,4.0,4.0,4.5,8.87,1
2,317.0,104.0,3.0,3.0,3.5,8.00,1
3,322.0,110.0,3.0,3.5,2.5,8.67,1
4,314.0,103.0,2.0,2.0,3.0,8.21,0
...,...,...,...,...,...,...,...
495,332.0,108.0,5.0,4.5,4.0,9.02,1
496,337.0,117.0,5.0,5.0,5.0,9.87,1
497,330.0,120.0,5.0,4.5,5.0,9.56,1
498,312.0,103.0,4.0,4.0,5.0,8.43,0


In [15]:
y

0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
495    0.87
496    0.96
497    0.93
498    0.73
499    0.84
Name: Chance of Admit, Length: 500, dtype: float64

In [16]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=25)

In [17]:
X_train.shape

(375, 7)

In [18]:
X_test.shape

(125, 7)

In [19]:
y_train.shape

(375,)

In [20]:
y_test.shape

(125,)

In [21]:
std_sca=StandardScaler()

In [22]:
X_train=std_sca.fit_transform(X_train)

In [23]:
X_test=std_sca.transform(X_test)

In [24]:
X_train.shape

(375, 7)

In [25]:
X_test.shape

(125, 7)

In [32]:
def objective(trail,data=X,target=y):
    train_x,test_x,train_y,test_y=train_test_split(data,target,test_size=0.25,random_state=30)
    param={
        'tree_method':'gpu_hist',
        'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
        'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),
        'colsample_bytree' :trail.suggest_categorical('colsample_bytree' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'subsample' :trail.suggest_categorical('subsample' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'learning_rate' : trail.suggest_categorical('learning_rate' , [.00001,.0003,.008,.02,.01,1,8]),
        'n_estimators' :3000, #300,3000,etc kitna b de do
        'max_depth' :trail.suggest_categorical('max_depth', [3,4,5,6,7,8,9,10,11,12]),
        'random_state' :trail.suggest_categorical('random_state' , [10,20,30 ,2000 , 3454,243123]),
        'min_child_weight' :trail.suggest_int('min_child_weight' ,1,200)
        }
    Xgb_reg_model=xgb.XGBRegressor(**param)
    Xgb_reg_model.fit(train_x,train_y,eval_set=[(test_x,test_y)],verbose=True)
    pred_xgb=Xgb_reg_model.predict(test_x)
    mse=mean_squared_error(test_y,pred_xgb)
    return mse

In [33]:
find_params=optuna.create_study()
find_params.optimize(objective,n_trials=10)
find_params.best_trial.params

[I 2023-07-31 06:58:59,374] A new study created in memory with name: no-name-dbe6fccb-a3a8-4cbd-8f41-6d84f06bca62


[0]	validation_0-rmse:0.25754
[1]	validation_0-rmse:0.25754
[2]	validation_0-rmse:0.25754
[3]	validation_0-rmse:0.25753
[4]	validation_0-rmse:0.25753
[5]	validation_0-rmse:0.25753
[6]	validation_0-rmse:0.25753
[7]	validation_0-rmse:0.25753
[8]	validation_0-rmse:0.25752
[9]	validation_0-rmse:0.25752
[10]	validation_0-rmse:0.25752
[11]	validation_0-rmse:0.25752
[12]	validation_0-rmse:0.25752
[13]	validation_0-rmse:0.25751
[14]	validation_0-rmse:0.25751
[15]	validation_0-rmse:0.25751
[16]	validation_0-rmse:0.25751
[17]	validation_0-rmse:0.25751
[18]	validation_0-rmse:0.25751
[19]	validation_0-rmse:0.25750
[20]	validation_0-rmse:0.25750
[21]	validation_0-rmse:0.25750
[22]	validation_0-rmse:0.25750
[23]	validation_0-rmse:0.25750
[24]	validation_0-rmse:0.25749
[25]	validation_0-rmse:0.25749
[26]	validation_0-rmse:0.25749
[27]	validation_0-rmse:0.25749
[28]	validation_0-rmse:0.25749
[29]	validation_0-rmse:0.25749
[30]	validation_0-rmse:0.25748
[31]	validation_0-rmse:0.25748
[32]	validation_0-

<ipython-input-32-7f7ff8e8b5bc>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
<ipython-input-32-7f7ff8e8b5bc>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),


[48]	validation_0-rmse:0.25745
[49]	validation_0-rmse:0.25745
[50]	validation_0-rmse:0.25745
[51]	validation_0-rmse:0.25744
[52]	validation_0-rmse:0.25744
[53]	validation_0-rmse:0.25744
[54]	validation_0-rmse:0.25744
[55]	validation_0-rmse:0.25744
[56]	validation_0-rmse:0.25744
[57]	validation_0-rmse:0.25743
[58]	validation_0-rmse:0.25743
[59]	validation_0-rmse:0.25743
[60]	validation_0-rmse:0.25743
[61]	validation_0-rmse:0.25743
[62]	validation_0-rmse:0.25742
[63]	validation_0-rmse:0.25742
[64]	validation_0-rmse:0.25742
[65]	validation_0-rmse:0.25742
[66]	validation_0-rmse:0.25742
[67]	validation_0-rmse:0.25742
[68]	validation_0-rmse:0.25741
[69]	validation_0-rmse:0.25741
[70]	validation_0-rmse:0.25741
[71]	validation_0-rmse:0.25741
[72]	validation_0-rmse:0.25741
[73]	validation_0-rmse:0.25740
[74]	validation_0-rmse:0.25740
[75]	validation_0-rmse:0.25740
[76]	validation_0-rmse:0.25740
[77]	validation_0-rmse:0.25740
[78]	validation_0-rmse:0.25739
[79]	validation_0-rmse:0.25739
[80]	val

[I 2023-07-31 06:59:09,767] Trial 0 finished with value: 0.06356059923060567 and parameters: {'lambda': 0.00122937815254677, 'alpha': 0.40885256788608715, 'colsample_bytree': 0.6, 'subsample': 0.6, 'learning_rate': 1e-05, 'max_depth': 8, 'random_state': 10, 'min_child_weight': 132}. Best is trial 0 with value: 0.06356059923060567.
<ipython-input-32-7f7ff8e8b5bc>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
<ipython-input-32-7f7ff8e8b5bc>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),


...

[0]	validation_0-rmse:0.25754
[1]	validation_0-rmse:0.25754
[2]	validation_0-rmse:0.25754
[3]	validation_0-rmse:0.25754
[4]	validation_0-rmse:0.25754
[5]	validation_0-rmse:0.25754
[6]	validation_0-rmse:0.25754
[7]	validation_0-rmse:0.25754
[8]	validation_0-rmse:0.25754
[9]	validation_0-rmse:0.25754
[10]	validation_0-rmse:0.25754
[11]	validation_0-rmse:0.25754
[12]	validation_0-rmse:0.25754
[13]	validation_0-rmse:0.25754
[14]	validation_0-rmse:0.25754
[15]	validation_0-rmse:0.25754
[16]	validation_0-rmse:0.25754
[17]	validation_0-rmse:0.25754
[18]	validation_0-rmse:0.25754
[19]	validation_0-rmse:0.25754
[20]	validation_0-rmse:0.25754
[21]	validation_0-rmse:0.25754
[22]	validation_0-rmse:0.25754
[23]	validation_0-rmse:0.25754
[24]	validation_0-rmse:0.25754
[25]	validation_0-rmse:0.25754
[26]	validation_0-rmse:0.25754
[27]	validation_0-rmse:0.25754
[28]	validation_0-rmse:0.25754
[29]	validation_0-rmse:0.25754
[30]	validation_0-rmse:0.25754
[31]	validation_0-rmse:0.25754
[32]	validation_0-

<ipython-input-32-7f7ff8e8b5bc>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
<ipython-input-32-7f7ff8e8b5bc>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),


[0]	validation_0-rmse:0.25754
[1]	validation_0-rmse:0.25754
[2]	validation_0-rmse:0.25754
[3]	validation_0-rmse:0.25753
[4]	validation_0-rmse:0.25753
[5]	validation_0-rmse:0.25753
[6]	validation_0-rmse:0.25753
[7]	validation_0-rmse:0.25753
[8]	validation_0-rmse:0.25752
[9]	validation_0-rmse:0.25752
[10]	validation_0-rmse:0.25752
[11]	validation_0-rmse:0.25752
[12]	validation_0-rmse:0.25752
[13]	validation_0-rmse:0.25751
[14]	validation_0-rmse:0.25751
[15]	validation_0-rmse:0.25751
[16]	validation_0-rmse:0.25751
[17]	validation_0-rmse:0.25751
[18]	validation_0-rmse:0.25751
[19]	validation_0-rmse:0.25750
[20]	validation_0-rmse:0.25750
[21]	validation_0-rmse:0.25750
[22]	validation_0-rmse:0.25750
[23]	validation_0-rmse:0.25750
[24]	validation_0-rmse:0.25749
[25]	validation_0-rmse:0.25749
[26]	validation_0-rmse:0.25749
[27]	validation_0-rmse:0.25749
[28]	validation_0-rmse:0.25749
[29]	validation_0-rmse:0.25749
[30]	validation_0-rmse:0.25748
[31]	validation_0-rmse:0.25748
[32]	validation_0-

[I 2023-07-31 07:00:42,549] Trial 9 finished with value: 0.06353655144875626 and parameters: {'lambda': 0.8632103195265196, 'alpha': 8.007261011074636, 'colsample_bytree': 0.4, 'subsample': 0.9, 'learning_rate': 1e-05, 'max_depth': 8, 'random_state': 3454, 'min_child_weight': 3}. Best is trial 2 with value: 0.020877083029907328.


{'lambda': 1.7553225699621027,
 'alpha': 4.278270209567875,
 'colsample_bytree': 0.4,
 'subsample': 0.6,
 'learning_rate': 0.01,
 'max_depth': 11,
 'random_state': 30,
 'min_child_weight': 152}

In [34]:
find_params.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_random_state,params_subsample,state
0,0,6.356060e-02,2023-07-31 06:58:59.377134,2023-07-31 06:59:09.767232,0 days 00:00:10.390098,0.408853,0.6,0.001229,0.00001,8,132,10,0.6,COMPLETE
1,1,6.347210e-02,2023-07-31 06:59:09.776397,2023-07-31 06:59:20.870044,0 days 00:00:11.093647,0.012970,0.1,9.525552,0.00001,10,67,30,0.5,COMPLETE
2,2,2.087708e-02,2023-07-31 06:59:20.874776,2023-07-31 06:59:30.725592,0 days 00:00:09.850816,4.278270,0.4,1.755323,0.01000,11,152,30,0.6,COMPLETE
3,3,6.632720e-02,2023-07-31 06:59:30.730711,2023-07-31 06:59:40.796546,0 days 00:00:10.065835,0.000246,0.8,0.001741,0.01000,9,94,30,0.1,COMPLETE
4,4,6.632720e-02,2023-07-31 06:59:40.802075,2023-07-31 06:59:50.256216,0 days 00:00:09.454141,0.000205,0.4,0.000225,0.00001,6,171,20,0.3,COMPLETE
5,5,6.632720e-02,2023-07-31 06:59:50.258751,2023-07-31 06:59:59.980210,0 days 00:00:09.721459,4.617636,0.4,0.842514,8.00000,7,98,243123,0.1,COMPLETE
6,6,2.804636e-02,2023-07-31 06:59:59.994137,2023-07-31 07:00:10.478802,0 days 00:00:10.484665,0.021344,0.6,5.496042,0.00030,4,91,30,0.3,COMPLETE
7,7,8.121652e+72,2023-07-31 07:00:10.484061,2023-07-31 07:00:20.727504,0 days 00:00:10.243443,0.167254,0.4,0.710444,8.00000,12,113,20,0.8,COMPLETE
8,8,1.893522e+17,2023-07-31 07:00:20.735586,2023-07-31 07:00:30.608391,0 days 00:00:09.872805,0.000152,0.4,0.006085,8.00000,3,138,20,0.3,COMPLETE
9,9,6.353655e-02,2023-07-31 07:00:30.610869,2023-07-31 07:00:42.548882,0 days 00:00:11.938013,8.007261,0.4,0.863210,0.00001,8,3,3454,0.9,COMPLETE


In [36]:
optuna.visualization.plot_optimization_history(find_params)

In [37]:
optuna.visualization.plot_slice(find_params)

In [38]:
optuna.visualization.plot_contour(find_params,params=['alpha','lambda'])

In [39]:
best_params={'lambda': 1.7553225699621027,
 'alpha': 4.278270209567875,
 'colsample_bytree': 0.4,
 'subsample': 0.6,
 'learning_rate': 0.01,
 'max_depth': 11,
 'random_state': 30,
 'min_child_weight': 152}

In [41]:
model=xgb.XGBRegressor(**best_params)

In [42]:
model.fit(X_train,y_train)

XGBRegressor(alpha=4.278270209567875, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.4, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=1.7553225699621027,
             learning_rate=0.01, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=11,
             max_leaves=None, min_child_weight=152, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, ...)

In [43]:
y_pred=model.predict(X_test)

In [49]:
#r2_score=[0,1]
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

-0.4274248896385675

In [50]:
from sklearn.ensemble import RandomForestRegressor
model2=RandomForestRegressor()
model2.fit(X_train,y_train)
y_pred2=model2.predict(X_test)
r2_score(y_test,y_pred2)

0.8154579977175695